In [1]:
import serial
import time

In [2]:
#初始化与红外相连的串口信息
port="COM10"
byts=9600
s=serial.Serial(port,byts)
#初始化与继电器相连的串口信息
port="COM9"
byts=9600
s1=serial.Serial(port,byts)

In [3]:
#接收红外信息，并传输给与继电器相连的PC,打开继电器，并读取继电器工作状态
infor=s.read(1)#读取第一个字节
print(infor.hex())
if infor.hex()!="":
    #打开继电器
    s1.flushInput()
    open=[0xFF,0x05,0x00,0x00,0xFF,0x00,0x99,0xE4]
    s1.write(bytes(open))
    #检查继电器是否执行打开命令
    recv=s1.read(8)
    recv_hex=recv.hex()
    print(recv_hex)
    if str(recv_hex)=="ff050000ff0099e4":
        print("执行命令正常")
    else:
        print("执行命令失误")
    time.sleep(2)
    #读取继电器状态
    s1.flushInput()
    state=[0xFF,0x01,0x00,0x00,0x00,0x08,0x28,0x12]
    s1.write(bytes(state))
    recv=s1.read(6)
    recv_hex=recv.hex()
    print("当前继电器状态为:{}".format("开" if recv[3] else '关'))
    time.sleep(2)
#将继电器执行状态反馈给红外发射端PC
if recv.hex()!="":
    s.flushInput()
    signal_to_send=[0xA1,0XF1,0x1C,0x2F,0x33]
    s.write(bytes(signal_to_send))
    time.sleep(2)

1c
ff050000ff0099e4
执行命令正常
当前继电器状态为:开


In [4]:
#接收红外信息，并传输给与继电器相连的PC,关闭继电器，并读取继电器工作状态
infor=s.read(1)#读取第一个字节
print(infor.hex())
if infor.hex()!="1":
    #关闭继电器
    s1.flushInput()
    close=[0xFF,0x05,0x00,0x00,0x00,0x00,0xD8,0x14]
    s1.write(bytes(close))
    #检查继电器是否执行关闭命令
    recv=s1.read(8)
    recv_hex=recv.hex()
    print(recv_hex)
    if str(recv_hex)=="ff0500000000d814":
        print("执行命令正常")
    else:
        print("执行命令失误")
    time.sleep(2)
    #将继电器执行状态反馈给红外发射端PC
    s.flushInput()
    signal_to_send=[0xA1,0XF1,0x1C,0x2F,0x33]
    s.write(bytes(signal_to_send))
    time.sleep(2)
    #读取继电器状态
    s1.flushInput()
    state=[0xFF,0x01,0x00,0x00,0x00,0x08,0x28,0x12]
    s1.write(bytes(state))
    recv=s1.read(6)
    recv_hex=recv.hex()
    print("当前继电器状态为:{}".format("开" if recv[3] else '关'))
    time.sleep(2)
#将继电器执行状态反馈给红外发射端PC
if not recv[3]:
    s.flushInput()
    signal_to_send=[0xA1,0XF1,0x1C,0x2F,0x33]
    s.write(bytes(signal_to_send))
    time.sleep(2)

1c
ff0500000000d814
执行命令正常
当前继电器状态为:关


In [5]:
#关闭串口
s.flushInput()
s.close()
s1.flushInput()
s1.close()